<a href="https://colab.research.google.com/github/david-levin11/Verification_Notebooks/blob/main/PlotHRRR_AK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Objective: To plot archived HRRR data for AK**
<br/>
Description--This script will attempt to use the Herbie python package to download and plot HRRR AK data for case reviews/operational recaps.

- David Levin, Arctic Testbed & Proving Ground, Anchorage Alaska

##**2 - Install and Import Packages**
This will take about a minute to run.

In [1]:
# @title
!pip install ecmwflibs
!pip install eccodes
!pip install cfgrib
!pip install curl
!pip install eccodes
!pip install wgrib2
!pip install herbie-data[extras]
!pip install ipywidgets
from herbie import Herbie
from herbie.toolbox import EasyMap, pc
from herbie import paint
try:
    import numpy as np
    import os
    from datetime import datetime, timedelta
    import matplotlib as mpl
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
except ImportError:
    raise ImportError("herbie.paint requires matplotlib.")
import ipywidgets as widgets
from IPython.display import display
from google.colab import output
output.enable_custom_widget_manager()

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 MB 8.8 MB/s eta 0:00:00
  Created wheel for findlibs: filename=findlibs-0.0.5-py3-none-any.whl size=6943 sha256=8f1590e59d631234c4389b854f3839dea7d47e3f1422d0e0aff8f1f91970f3e9
  Stored in directory: /root/.cache/pip/wheels/2e/67/14/22fa5b9fd9c41be520b37e908597d6a262803c0fcf2ba7c2c3
Successfully built findlibs
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 675.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 886.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 1.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement curl (from versions: none)
ERROR: No matching distribution found for curl
ERROR: Could not find a version that satisfies the requirement wgrib2 (from versions: none)
ERROR: No matching distribution found for wgrib2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 57.9 MB/s eta 0:00

##**3 - Download & Plot**

In [9]:
###################### Classes for Colormaps ##################################
# setting up our colorscales for plotting precip since Herbie only uses mm    #
# can use paint for everything else                                           #
###############################################################################
def make_custom_cmaps(name, colors, bounds: list = None, N: int = None):
    if N is None:
        N = len(colors)
    linear_cmap = mcolors.LinearSegmentedColormap.from_list(name, colors)
    segment_cmap = mcolors.LinearSegmentedColormap.from_list(name + "2", colors, N=N)

    # When data is NaN, set color to transparent
    linear_cmap.set_bad("#ffffff00")
    segment_cmap.set_bad("#ffffff00")

    for cm in [linear_cmap, segment_cmap]:
        mpl.colormaps.register(cmap=cm, force=True)
        mpl.colormaps.register(cmap=cm.reversed(), force=True)

    if bounds is not None:
        return (
            mcolors.Normalize(bounds.min(), bounds.max()),
            mcolors.BoundaryNorm(bounds, linear_cmap.N),
        )

class NWSPrecipitation:
    """National Weather Service precipitation amount colorbar properties.

    Also known as Qualitative Precipitation Forecast/Estimate (QPF/QPE).
    """

    name = "nws.pcp"
    units = "in"
    variable = "Precipitation"
    colors = np.array(
        [
            "#ffffff",
            "#c7e9c0",
            "#a1d99b",
            "#74c476",
            "#31a353",
            "#006d2c",
            "#fffa8a",
            "#ffcc4f",
            "#fe8d3c",
            "#fc4e2a",
            "#d61a1c",
            "#ad0026",
            "#700026",
            "#3b0030",
            "#4c0073",
            "#ffdbff",
        ]
    )
    # NWS bounds in inches
    bounds = np.array(
        [0, 0.01, 0.1, 0.25, 0.5, 1, 1.5, 2, 3, 4, 6, 8, 10, 15, 20, 30, 50]
    )
    norm, norm2 = make_custom_cmaps(name, colors, bounds)
    cmap = plt.get_cmap(name)
    cmap2 = plt.get_cmap(name + "2")
    kwargs = dict(cmap=cmap, norm=norm)
    kwargs2 = dict(cmap=cmap, norm=norm2)
    cbar_kwargs = dict(label=f"{variable} ({units})")
    cbar_kwargs2 = cbar_kwargs | dict(spacing="uniform", ticks=bounds)

class NWSWindSpeed:
    name = "nws.wind"
    units = r"mph"
    variable = "Wind Speed"
    colors = np.array(
        [
            "#103f78",
            "#225ea8",
            "#1d91c0",
            "#41b6c4",
            "#7fcdbb",
            "#b4d79e",
            "#dfff9e",
            "#ffffa6",
            "#ffe873",
            "#ffc400",
            "#ffaa00",
            "#ff5900",
            "#ff0000",
            "#a80000",
            "#6e0000",
            "#ffbee8",
            "#ff73df",
        ]
    )
    # MPH
    bounds = np.array(
        [0.0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 100, 120, 140, 160]
    )
    norm, norm2 = make_custom_cmaps(name, colors, bounds)
    cmap = plt.get_cmap(name)
    cmap2 = plt.get_cmap(name + "2")
    kwargs = dict(cmap=cmap, norm=norm)
    kwargs2 = dict(cmap=cmap, norm=norm2)
    cbar_kwargs = dict(label=f"{variable} ({units})")
    cbar_kwargs2 = cbar_kwargs | dict(spacing="proportional", ticks=bounds)
'''
# Get a list of all colormap names
colormaps = plt.colormaps()

# Print each colormap name
for cmap in colormaps:
    print(cmap)
'''
###########################################################################
##################### Config ##############################################

model = "hrrrak"
timestep = 3
#runlength = 48
#@markdown Choose Model Run Date
rundate = "2024-06-27" #@param {type:"date"}
#@markdown Choose Model Run Hour (UTC)
hour = 12 #@param {type:"slider", min:0, max:21, step:3}

runhour = str(hour).zfill(2)

runstring = rundate + " " +runhour + ":00"

run_time = rundate + " " +runhour + "Z"

#@markdown Choose your forecast start hour
start_hour = 6 #@param {type:"slider", min:0, max:48, step:3}

#@markdown Choose your forecast end hour
end_hour = 6 #@param {type:"slider", min:0, max:48, step:3}

# Ensure end_hour is greater than start_hour
if end_hour < start_hour:
    end_hour = start_hour

# Generate the list of forecast hours from start to end with a step of 3
forecast_hours = list(range(start_hour, end_hour + 1, timestep))
print(f"Selected forecast hour range: {start_hour} to {end_hour}")
print(f"Forecast hours list: {forecast_hours}")

#@markdown Choose your weather element to plot (current choices are: 6hrPrecip, 1hrPrecip, 2mTemp, Reflectivity, 10mWind)
element = "10mWind" #@param {type:"string"}

#@markdown If you're plotting wind, choose your thinning factor for wind barbs (50 is a good number for the full AK domain).  For small domains try a smaller number like 10 or 20.
thin_factor = 50 #@param {type:"integer"}

element_list = ["6hrPrecip","1hrPrecip","2mTemp","Reflectivity", "10mWind"]

if element not in element_list:
  raise ValueError(f"Element must be one of {element_list}")

product = "sfc"

#@markdown Would you like to have a custom zoom?  If so, check the box and enter appropriate values for lat/lons below
#@markdown Default is all of AK
custom_extent = False #@param {type:"boolean"}

if custom_extent:
  west = -171.00 #@param {type:"number"}
  north = 72.00 #@param {type:"number"}
  east = -149.00 #@param {type:"number"}
  south = 65.00 #@param {type:"number"}

graphicsdir = '/nas/hrrr/graphics'
################################# Main Code ################################
for fcst_hr in forecast_hours:
  valid_time = (datetime.strptime(runstring, "%Y-%m-%d %H:%M") + timedelta(hours=fcst_hr)).strftime("%Y-%m-%d %HZ")
  print(f"Now working on {model} {element} for valid time of {valid_time}")
  # logic for selecting variables
  if element == "1hrPrecip":
    # getting the correct run based on inputs
    H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
    ds = H.xarray(":APCP:.*:(?:0-1|[1-9]\d*-\d+) hour")
    oldvar = ds.tp/25.4
    var = oldvar.where(oldvar>0.009)
    name = f"1hr {ds.tp.GRIB_name.split(' ')[-1]}"
    description = ds.description
    kwargs = NWSPrecipitation.kwargs2
    cbar_kwargs = NWSPrecipitation.cbar_kwargs2

  elif element == "6hrPrecip":
    try:
    #have to subtract the previous total precip accum from the current
    #This won't work unless we have both time steps
      H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
      H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr-6)
      ds = H.xarray(":APCP:surface:0-[1-9]*")
      ds2 = H2.xarray(":APCP:surface:0-[1-9]*")
      late = ds.tp/25.4
      early = ds2.tp/25.4
      oldvar = late-early
      var = oldvar.where(oldvar>0.009)
      name = f"6hr {ds.tp.GRIB_name.split(' ')[-1]}"
      description = ds.description
      kwargs = NWSPrecipitation.kwargs2
      cbar_kwargs = NWSPrecipitation.cbar_kwargs2

    except Exception as e:
      print(e)
      continue
  elif element == "2mTemp":
    H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
    ds = H.xarray(":TMP:2 m above")
    var = ds.t2m-273.15
    name = ds.tp.GRIB_name
    description = ds.product_description
    kwargs = paint.NWSTemperature.kwargs2
    cbar_kwargs = paint.NWSTemperature.cbar_kwargs2

  elif element == "Reflectivity":
    H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
    ds = H.xarray(":REFC:")
    var = ds.refc.where(ds.refc>0)
    name = ds.refc.GRIB_name
    description = ds.description
    #kwargs = paint.NWSRefectivity.kwargs2
    #cbar_kwargs = paint.NWSReflectivity.cbar_kwargs2

  elif element == "10mWind":
    H = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
    ds = H.xarray(":UGRD:10 m")
    H2 = Herbie(runstring, model=model, product=product, fxx=fcst_hr)
    ds2 = H2.xarray(":VGRD:10 m")
    # creating speed and direction products
    mag = np.sqrt(ds.u10**2 + ds2.v10**2)
    name = "10m Wind"
    description = ds.description
    kwargs = NWSWindSpeed.kwargs2
    cbar_kwargs = NWSWindSpeed.cbar_kwargs2
    var = mag*2.23694 #converting to mph
    # Thin out the wind barb data by slicing
    u_thin = ds.u10[::thin_factor, ::thin_factor]*2.23694
    v_thin = ds2.v10[::thin_factor, ::thin_factor]*2.23694
    lon_thin = ds.longitude[::thin_factor, ::thin_factor]
    lat_thin = ds.latitude[::thin_factor, ::thin_factor]

  modelname = ds.model.upper()

  #print(ds)
  if "Precip" in element:
    print(f"{ds.tp.GRIB_stepRange=}")
    try:
      print(f"{ds2.tp.GRIB_stepRange=}")
    except:
      print("No second step range found")

  ax = EasyMap("50m", crs=ds.herbie.crs, figsize=[10,8]).STATES().OCEAN().LAND().ax

  # setting custom extent if necessary
  if custom_extent:
    ax.set_extent([west, east, south, north])
  if element != "Reflectivity":
    #plotting magnitude fields
    p = ax.pcolormesh(ds.longitude, ds.latitude, var, transform=pc, **kwargs)
    # plotting wind barbs if wind
    if element == "10mWind":
      ax.barbs(lon_thin.values, lat_thin.values, u_thin.values, v_thin.values, length=6, transform=pc)

    plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.05, **cbar_kwargs)

    ax.set_title(f"{modelname}--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)

  else:
    p = ax.pcolormesh(ds.longitude, ds.latitude, var, transform=pc, cmap='radar.reflectivity', vmin=0, vmax=70)

    plt.colorbar(p, ax=ax, orientation="horizontal", pad=0.05)

    ax.set_title(f"{modelname}--Run: {run_time}\n{name} Valid {valid_time}", loc="center", pad=5)

  # setting the title for our graphic
  graphictitle = f'{modelname}_{element}_{run_time.replace("-","").replace(" ","")}_{valid_time.replace("-","").replace(" ","")}.png'

  if not os.path.exists(graphicsdir):
    os.makedirs(graphicsdir)
  plt.savefig(f"{graphicsdir}/{graphictitle}")
  print(f"Saved {graphictitle} to {graphicsdir}")
  plt.close()


<ipython-input-9-db2b9d37cae5>:16: UserWarning: Overwriting the cmap 'nws.pcp' that was already in the registry.
  mpl.colormaps.register(cmap=cm, force=True)
<ipython-input-9-db2b9d37cae5>:17: UserWarning: Overwriting the cmap 'nws.pcp_r' that was already in the registry.
  mpl.colormaps.register(cmap=cm.reversed(), force=True)
<ipython-input-9-db2b9d37cae5>:16: UserWarning: Overwriting the cmap 'nws.pcp2' that was already in the registry.
  mpl.colormaps.register(cmap=cm, force=True)
<ipython-input-9-db2b9d37cae5>:17: UserWarning: Overwriting the cmap 'nws.pcp2_r' that was already in the registry.
  mpl.colormaps.register(cmap=cm.reversed(), force=True)
<ipython-input-9-db2b9d37cae5>:16: UserWarning: Overwriting the cmap 'nws.wind' that was already in the registry.
  mpl.colormaps.register(cmap=cm, force=True)
<ipython-input-9-db2b9d37cae5>:17: UserWarning: Overwriting the cmap 'nws.wind_r' that was already in the registry.
  mpl.colormaps.register(cmap=cm.reversed(), force=True)
<ip

Selected forecast hour range: 6 to 6
Forecast hours list: [6]
Now working on hrrrak 10mWind for valid time of 2024-06-27 18Z
✅ Found ┊ model=hrrrak ┊ product=sfc ┊ 2024-Jun-27 12:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrrak ┊ product=sfc ┊ 2024-Jun-27 12:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws
Saved HRRRAK_10mWind_2024062712Z_2024062718Z.png to /nas/hrrr/graphics
